In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import MELODIC
%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects_sim.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [3]:
def apply_melodic(in_tuple):
    sub = in_tuple[0]
    exp = in_tuple[1]
    run = str(in_tuple[2])
    
    sub_path = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + run + '/'
    
    if os.path.exists(sub_path):
        scan = sub_path + 'unsmoothed_timeseries.nii.gz'
        mask = sub_path + 'functional_mask.nii.gz'
        out_dir = sub_path + 'melodic'

        if not os.path.exists(out_dir):
            os.mkdir(out_dir)

        melodic_setup = MELODIC(in_files = scan, no_bet = True, bg_threshold = 3, tr_sec = 1.5,
                               mm_thresh = .5, out_stats = True, report = True, mask = mask,
                               out_dir = out_dir)
        melodic_setup.run()

In [4]:
in_tuples = []
for sub in sub_list:
    for exp in ['sim']:
        for run in range(1,4):
            in_tuples.append((sub,exp,run))

pool = multiprocessing.Pool(processes = 24)
pool.map(apply_melodic,in_tuples)

FileNotFoundError: File/Directory '/data/home/iballard/fd/report' not found for MELODIC output 'report_dir'.
Interface MELODIC failed to run. 

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running in parallel.
INFO:workflow:Submitting 2 jobs
INFO:workflow:Executing: mask ID: 0
INFO:workflow:Executing: median ID: 2
INFO:workflow:[Job finished] jobname: median jobid: 2
INFO:workflow:Executing node mask in dir: /tmp/tmpMPqlwH/smooth/susan_smooth/mask
INFO:workflow:Executing node median in dir: /tmp/tmp2RwedY/smooth/susan_smooth/median
INFO:workflow:Executing node _mask0 in dir: /tmp/tmpMPqlwH/smooth/susan_smooth/mask/mapflow/_mask0
INFO:workflow:Executing node _median0 in dir: /tmp/tmp2RwedY/smooth/susan_smooth/median/mapflow/_median0
INFO:workflow:Running: fslmaths /data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/melodic/filtered_func_data_clean.nii.gz -mas /data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/functional_mask.nii.gz /tmp/tmpMPqlwH/smooth/susan_smooth/mask/mapflow/_mask0/filtered_func_data_clean_mask.nii.gz
INFO:workflow:Running: fslstats /data/home/iballard/fd/analysis/ser_8mm